In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime

In [2]:
spark = SparkSession.builder.master('local[*]').appName('hw_2025').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/13 08:17:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 08:17:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/13 08:17:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Question 1: Install Spark and PySpark
# Execute spark.version

spark.version

#### Answer: 3.3.2

'3.3.2'

In [4]:
# Question 2: Yellow October 2024
# Read the October 2024 Yellow into a Spark Dataframe.
# Repartition the Dataframe to 4 partitions and save it to parquet.
# What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? 
# Select the answer which most closely matches.

!curl -Lf https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -o hw_2025.parquet
df = spark.read.parquet('hw_2025.parquet')
df.repartition(4).write.parquet('hw_2025')
!ls -lash hw_2025/
    
#### Answer: 25 MB

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.3M  100 61.3M    0     0  11.6M      0  0:00:05  0:00:05 --:--:-- 14.2M


[Stage 3:>                                                          (0 + 4) / 4]

total 98M
4.0K drwxr-xr-x 2 ajay ajay 4.0K Jan 13 08:17 .
4.0K drwxrwxr-x 4 ajay ajay 4.0K Jan 13 08:17 ..
4.0K -rw-r--r-- 1 ajay ajay    8 Jan 13 08:17 ._SUCCESS.crc
196K -rw-r--r-- 1 ajay ajay 194K Jan 13 08:17 .part-00000-f8d47746-8862-4720-ae84-751d9f7cd483-c000.snappy.parquet.crc
196K -rw-r--r-- 1 ajay ajay 194K Jan 13 08:17 .part-00001-f8d47746-8862-4720-ae84-751d9f7cd483-c000.snappy.parquet.crc
196K -rw-r--r-- 1 ajay ajay 194K Jan 13 08:17 .part-00002-f8d47746-8862-4720-ae84-751d9f7cd483-c000.snappy.parquet.crc
196K -rw-r--r-- 1 ajay ajay 195K Jan 13 08:17 .part-00003-f8d47746-8862-4720-ae84-751d9f7cd483-c000.snappy.parquet.crc
   0 -rw-r--r-- 1 ajay ajay    0 Jan 13 08:17 _SUCCESS
 25M -rw-r--r-- 1 ajay ajay  25M Jan 13 08:17 part-00000-f8d47746-8862-4720-ae84-751d9f7cd483-c000.snappy.parquet
 25M -rw-r--r-- 1 ajay ajay  25M Jan 13 08:17 part-00001-f8d47746-8862-4720-ae84-751d9f7cd483-c000.snappy.parquet
 25M -rw-r--r-- 1 ajay ajay  25M Jan 13 08:17 part-00002-f8d47746-8862-472

In [5]:
# Question 3: Count records
# How many taxi trips were there on the 15th of October?
# Consider only trips that started on the 15th of October.

df.createTempView('hw')
spark.sql(("""
            
            select 
                count(*) as num_trips 
            from hw
            where date(tpep_pickup_datetime) = '2024-10-15'
            
            """)).show()

#### Answer: 128893

[Stage 4:>                                                          (0 + 4) / 4]

+---------+
|num_trips|
+---------+
|   128893|
+---------+



In [6]:
# Question 4: Longest trip
# What is the length of the longest trip in the dataset in hours?

spark.sql(("""
           
            select 
                date(tpep_pickup_datetime) as date, 
                round(max(
                    (unix_timestamp(tpep_dropoff_datetime) - 
                    unix_timestamp(tpep_pickup_datetime)) 
                    / 3600.0), 2) as longest
            from hw
            group by 1
            order by 2 desc
            limit 5
            
            """)).show()

#### Answer: 162.62

[Stage 7:============================================>              (3 + 1) / 4]

+----------+-------+
|      date|longest|
+----------+-------+
|2024-10-16| 162.62|
|2024-10-03| 143.33|
|2024-10-22| 137.76|
|2024-10-18| 114.83|
|2024-10-21|  89.90|
+----------+-------+



In [7]:
# Question 5: User Interface
# Spark’s User Interface which shows the application's dashboard runs on which local port?

df._session

#### Answer: Usually 4040 but mine is running on 4042 since 4040 is occupied.

In [8]:
# Question 6: Least frequent pickup location zone
# Load the zone lookup data into a temp view in Spark:
# wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -o z.csv
zone = spark.read.option('header', 'true').option('inferSchema', 'true').csv('z.csv')
joined = df.join(zone, zone.LocationID == df.PULocationID, how = 'left')
joined.createOrReplaceTempView('hw')

spark.sql(("""
            
            select Zone
            from hw
            group by 1
            order by count(*)
            
            """)).head(1)

#### Answer: Governor's Island/Ellis Island/Liberty Island

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  22799      0 --:--:-- --:--:-- --:--:-- 22799


[Row(Zone="Governor's Island/Ellis Island/Liberty Island")]

In [9]:
spark.stop()
!rm -rf *.csv *.parquet hw_*